In [9]:
import tweepy
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from datetime import datetime
from config import conkey, conkeysecret, atoken, atokensecret
import pandas as pd
import numpy as np

In [13]:
analyzer = SentimentIntensityAnalyzer()
auth = tweepy.OAuthHandler(conkey,conkeysecret)
auth.set_access_token(atoken,atokensecret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser(),wait_on_rate_limit=True)

In [14]:
stock_data = pd.read_csv("StocksDow.csv")
stock_data.head()

,Symbol,Company
0,AXP,American Express Co
1,AAPL,Apple Inc
2,BA,Boeing Co
3,CAT,Caterpillar Inc
4,CSCO,Cisco Systems Inc


In [12]:
#pulled from whatever source we're using, set as lists for now, but can be changed
pull_dates = ["2018-04-04","2018-04-03","2018-04-02","2018-04-01","2018-03-31","2018-03-30","2018-03-29"]

#for dataframe
stock_name_daily = []
sentiments_daily = []

prev_date = "2018-04-05"
#loop to check for every stock for every date and grab a sample of 100 tweets
for stocks in stock_data['Company']:
    for dates in pull_dates:
        stock_text = []
        sentiments = []
        tweets = api.search(q=stock_name,count=100,since=dates,until=prev_date)
        prev_date = dates
        #run through 100 tweets for that day
        for public_tweets in tweets['statuses']:
            stock_text.append(public_tweets['text'])
            #vader sentiments
            sentiments.append(analyzer.polarity_scores(public_tweets['text'])['compound'])
            #append daily data to tables
        sentiments_daily.append(np.mean(sentiments))
        stock_name_daily.append(stocks)
                

#Setting Up Dataframe                
twitter_df = pd.DataFrame({'Stock Name':stock_name_daily,
                            'Date':pull_dates,
                            'Sentiments':sentiments_daily})
twitter_df.head()

C:\Users\conle\Anaconda3\envs\PythonData\lib\site-packages\numpy\core\fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\conle\Anaconda3\envs\PythonData\lib\site-packages\numpy\core\_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


RateLimitError: [{'message': 'Rate limit exceeded', 'code': 88}]